# Toronto Neighborhood Scraping 
### Import all necessary libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import geocoder 
import os

### Access Wikipedia site and import into BeautifulSoup object

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text

soup = BeautifulSoup(website_url)

### Read Soup boject into dataframe

In [3]:
table_contents=[]
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### Inspect dataframe 

In [4]:
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### dataframe Shape

In [5]:
df.shape

(103, 3)

### geocoding

In [6]:
#postal_code = "M3A"
#lat_lng_coords = None

#loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude= lat_lng_coords[1]

In [7]:
file_location = "C:\\Users\\andre.mercanzini\\Documents\\Python Scripts"
basefile_name = "Geospatial_Coordinates.csv"
file_name = os.path.join(file_location, basefile_name)
print(file_name)
dfGEO = pd.read_csv(file_name)

C:\Users\andre.mercanzini\Documents\Python Scripts\Geospatial_Coordinates.csv


In [8]:
#df.assign(ColName = 'Latitude')
#df.assign(ColName ='Longitude')
Lat_list = []
Long_list = []

for index, row in df.iterrows():
    GEOindex = (dfGEO[dfGEO['Postal Code']==row['PostalCode']].index.values)
    Lat = dfGEO._get_value(int(GEOindex), 'Latitude')
    Long = dfGEO._get_value(int(GEOindex), 'Longitude')
    Lat_list.append(Lat)
    Long_list.append(Long)
    #df._set_value(index, 'Latitude', Lat)
    #df._set_value(index, 'Longitude', Long)
    
df.insert(loc = 3, column='Latitude', value=Lat_list)
df.insert(loc = 4, column='Longitude', value=Long_list)    

In [9]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
